In [5]:
from sage.all import *
import numpy as np
class DecodingGoppaCode:
    """Implementation of the Patterson Algorithm to decode Goppa codes."""

    def __init__(self, m, t, ciphertext, G):
        self.m, self.t = m, t
        self.F = GF(2**m, 'z')
        R = PolynomialRing(self.F, 'x')
        self.x = R.gen()
        self.y = ciphertext
        self.Polynomial = self.x**2 + self.x + self.F.gen()**3
        self.Support = list(self.F)
        self.G = G

    def compute_inverse(self, value):
        """Compute the inverse of a polynomial modulo a given polynomial."""
        try:
            inverse = value.inverse_mod(self.Polynomial)
            verify = (inverse * value) % self.Polynomial
            if verify == 1:
                return inverse, True
        except ZeroDivisionError:
            return None, False
        return None, False

    def compute_rho(self, value):
        """Compute the rho function."""
        aux1 = (value + self.x) % self.Polynomial
        return aux1 ** (2**(self.m * self.t - 1)) % self.Polynomial

    def finding_ax_bx(self, Sx):
        """Find the polynomials a(x) and b(x)."""
        R = PolynomialRing(self.F, 'x')
        bx = R(1)  # Initial guess for b(x)
        Sx_inverse, valid = self.compute_inverse(Sx)
        if not valid:
            return None, None
        rho_Sx_inverse = self.compute_rho(Sx_inverse)
        ax = (bx * rho_Sx_inverse) % self.Polynomial
        if ax.degree() <= floor(self.t):
            return ax, bx
        else:
            return None, None

    def sigma(self, ax, bx):
        """Compute sigma(x)."""
        return (ax**2) + self.x * (bx**2)

    def root_sigma(self, sigma_x):
        """Find the roots of sigma(x) and determine error positions."""
        error_position = [0] * len(self.y)
        aux_roots = sigma_x.roots()
        roots = [value[0] for value in aux_roots]
        for i in roots:
            if i in self.Support:
                index_error = self.Support.index(i)
                error_position[index_error] = 1
        return error_position

    def find_codeword(self, Sx):
        """Compute the corrected codeword."""
        ax, bx = self.finding_ax_bx(Sx)
        if ax is None or bx is None:
            return None
        sigma_x = self.sigma(ax, bx)
        error = self.root_sigma(sigma_x)
        return np.bitwise_xor(self.y, error)

    def syndromes(self):
        """Compute the list of syndromes."""
        return [self.x - self.Support[i] for i, bit in enumerate(self.y) if bit != 0]

    def codeword_recovered(self):
        """Recover the codeword."""
        List_Sx = [
            self.x - 1, self.x - self.F.gen(), self.x - self.F.gen()**2,
            self.x - self.F.gen()**3, self.x - self.F.gen()**9, self.x - self.F.gen()**10,
            self.x - self.F.gen()**13, self.x - self.F.gen()**14
        ]
        List = [self.compute_inverse(item)[0] for item in List_Sx if self.compute_inverse(item)[1]]
        Sx = sum(List)
        return self.find_codeword(Sx)

    def verify_information_set(self, G):
        """Verify the information set I."""
        I = [col for col in range(G.shape[1]) if list(G[:, col]).count(1) == 1]
        return sorted(I)

    def message_recovered(self):
        """Recover the original message."""
        codeword_recovered = self.codeword_recovered()
        if codeword_recovered is None:
            return None
        G_I = self.G[:, np.array(self.verify_information_set(self.G))]
        inverse_G = np.mod(np.linalg.inv(G_I), 2).astype(int)
        codeword_I = [codeword_recovered[i] for i in self.verify_information_set(self.G)]
        message = np.mod(np.dot(codeword_I, inverse_G), 2)
        return message

# Example usage:
ciphertext, m, t = [0, 1, 1, 1, 1, 0, 0, 0, 0, 0, 1, 1, 0, 0, 1, 1], 4, 2
G = np.array([
    [1, 0, 1, 1, 0, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0],
    [1, 0, 0, 1, 0, 1, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0],
    [1, 1, 1, 0, 0, 1, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0],
    [0, 1, 0, 1, 1, 1, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0],
    [0, 0, 1, 0, 1, 1, 1, 0, 1, 0, 0, 0, 1, 0, 0, 0],
    [0, 1, 0, 0, 1, 1, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0],
    [1, 0, 1, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0],
    [1, 1, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1]
])
PattersonAlgorithm = DecodingGoppaCode(m, t, ciphertext, G)
message = PattersonAlgorithm.message_recovered()
print("Message:", message)


Message: [0 0 1 1 0 0 1 1]
